In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12001857887203449804,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11962352970175526340
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695549568
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12406747416933898431
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12273553035776734980
 physical_device_desc: "device: XLA_GPU device"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.4.3
tensorflow Version 2.3.0


In [5]:
batch_size = 32
img_height, img_width = 800, 600
input_shape = (img_height, img_width, 3)
epochs = 1000

In [7]:
print(os.listdir("../input/data-pig/"))

['test', 'train']


In [8]:
train_dir = '../input/data-pig/train/'
test_dir = '../input/data-pig/test/'

In [9]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [10]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
#     rotation_range=90,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.3,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='reflect',
    validation_split= 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 8061 images belonging to 4 classes.
Found 2012 images belonging to 4 classes.
Found 1776 images belonging to 4 classes.


In [11]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 8061
nb_validation_samples: 2012
nb_test_samples: 1776

predict_size_train: 252
predict_size_validation: 63
predict_size_test: 56

 num_classes: 4


In [12]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "VGG19_descriptors"

In [17]:
vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"
xception_weights = "../input/full-keras-pretrained-no-top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [18]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

In [19]:
input_tensor = Input(shape = input_shape)  

base_model1=Xception(weights=xception_weights, include_top=False, pooling = "avg")

In [20]:
# base_model1.summary()

In [21]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [22]:
bottleneck_final_model = base_model1

In [23]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [24]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [25]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [26]:

# import tensorflow as tf
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))

# reset_keras_tf_session()

In [27]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [28]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        'hidden1':hp.choice('hidden1',[4096,2048,1024,512,256]),
#         'hidden2':hp.choice('hidden2',[2048,1024,512,256,128,]),
#         'hidden3':hp.choice('hidden3',[1024,512,256,128,64,32]),    
        'bias_reg': hp.choice('bias_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'act_reg': hp.choice('act_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'ker_reg': hp.choice('ker_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'activation_function':hp.choice('activation_function',["relu","elu","selu","softplus","tanh",])
        }

In [29]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
#     dropout_rate = 0.5
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(params["hidden1"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
#     model.add(Dropout(0.25))

#     model.add(Dense(params["hidden2"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    
#     model.add(Dense(params["hidden3"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
    
    
    model.add(Dropout(0.5))
    

    model.add(Dense(num_classes, activation="softmax"))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [30]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [31]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:                                    
{'act_reg': 0.001, 'activation_function': 'tanh', 'beta_1': 0.7, 'beta_2': 0.999, 'bias_reg': 0.1, 'hidden1': 4096, 'ker_reg': 0.001, 'lr': 0.001}
Epoch 1/1000                                           
252/252 - 2s - loss: 2.1037 - accuracy: 0.9436 - val_loss: 0.9829 - val_accuracy: 0.9811

Epoch 2/1000                                           
252/252 - 1s - loss: 0.4409 - accuracy: 0.9996 - val_loss: 0.2865 - val_accuracy: 0.9970

Epoch 3/1000                                           
252/252 - 1s - loss: 0.5906 - accuracy: 0.9866 - val_loss: 31.9243 - val_accuracy: 0.3802

Epoch 4/1000                                           
252/252 - 1s - loss: 1.3561 - accuracy: 0.9706 - val_loss: 0.3260 - val_accuracy: 0.9886

Epoch 5/1000                                           
252/252 - 1s - loss: 0.7387 - accuracy: 0.9758 - val_loss: 0.2205 - val_accuracy: 0.9930

Epoch 6/1000                                           
252/252 - 

252/252 - 1s - loss: 0.2321 - accuracy: 0.9965 - val_loss: 0.1890 - val_accuracy: 0.9940

Epoch 56/1000                                          
252/252 - 1s - loss: 0.1751 - accuracy: 0.9995 - val_loss: 0.1767 - val_accuracy: 0.9955

Epoch 57/1000                                          
252/252 - 1s - loss: 0.1781 - accuracy: 0.9995 - val_loss: 0.1813 - val_accuracy: 0.9975

Epoch 58/1000                                          
252/252 - 1s - loss: 0.1723 - accuracy: 0.9999 - val_loss: 0.1834 - val_accuracy: 0.9945

Epoch 59/1000                                          
252/252 - 1s - loss: 1.2313 - accuracy: 0.9576 - val_loss: 0.3209 - val_accuracy: 0.9965

Epoch 60/1000                                          
252/252 - 1s - loss: 0.2657 - accuracy: 0.9979 - val_loss: 0.2207 - val_accuracy: 0.9970

Epoch 61/1000                                          
252/252 - 1s - loss: 0.2970 - accuracy: 0.9954 - val_loss: 0.2123 - val_accuracy: 0.9955

Epoch 62/1000                     

252/252 - 1s - loss: 0.2299 - accuracy: 0.9981 - val_loss: 0.2097 - val_accuracy: 0.9965

Epoch 112/1000                                         
252/252 - 1s - loss: 0.2073 - accuracy: 0.9983 - val_loss: 0.2031 - val_accuracy: 0.9970

Epoch 113/1000                                         
252/252 - 1s - loss: 0.1835 - accuracy: 0.9998 - val_loss: 0.1838 - val_accuracy: 0.9975

Epoch 114/1000                                         
252/252 - 1s - loss: 0.1724 - accuracy: 1.0000 - val_loss: 0.1889 - val_accuracy: 0.9960

Epoch 115/1000                                         
252/252 - 1s - loss: 0.1738 - accuracy: 1.0000 - val_loss: 0.1803 - val_accuracy: 0.9970

Epoch 116/1000                                         
252/252 - 1s - loss: 0.1739 - accuracy: 1.0000 - val_loss: 0.1979 - val_accuracy: 0.9906

Epoch 117/1000                                         
252/252 - 1s - loss: 1.0831 - accuracy: 0.9596 - val_loss: 0.5856 - val_accuracy: 0.9940

Epoch 118/1000                    

252/252 - 1s - loss: 0.1845 - accuracy: 0.9999 - val_loss: 0.2005 - val_accuracy: 0.9955

Epoch 168/1000                                         
252/252 - 1s - loss: 0.1840 - accuracy: 1.0000 - val_loss: 0.1960 - val_accuracy: 0.9970

Epoch 169/1000                                         
252/252 - 1s - loss: 2.2140 - accuracy: 0.9294 - val_loss: 1.1485 - val_accuracy: 0.9742

Epoch 170/1000                                         
252/252 - 1s - loss: 1.0787 - accuracy: 0.9735 - val_loss: 0.5504 - val_accuracy: 0.9796

Epoch 171/1000                                         
252/252 - 1s - loss: 0.3959 - accuracy: 0.9893 - val_loss: 0.5986 - val_accuracy: 0.9935

Epoch 172/1000                                         
252/252 - 1s - loss: 0.2966 - accuracy: 0.9954 - val_loss: 0.4429 - val_accuracy: 0.9911

Epoch 173/1000                                         
252/252 - 1s - loss: 0.2479 - accuracy: 0.9965 - val_loss: 0.2403 - val_accuracy: 0.9975

Epoch 174/1000                    

252/252 - 1s - loss: 0.1932 - accuracy: 0.9995 - val_loss: 0.1806 - val_accuracy: 0.9960

Epoch 224/1000                                         
252/252 - 1s - loss: 0.1805 - accuracy: 1.0000 - val_loss: 0.1887 - val_accuracy: 0.9960

Epoch 225/1000                                         
252/252 - 1s - loss: 1.2767 - accuracy: 0.9571 - val_loss: 4.5340 - val_accuracy: 0.9950

Epoch 226/1000                                         
252/252 - 1s - loss: 1.4224 - accuracy: 0.9772 - val_loss: 0.3825 - val_accuracy: 0.9901

Epoch 227/1000                                         
252/252 - 1s - loss: 0.4626 - accuracy: 0.9885 - val_loss: 0.2432 - val_accuracy: 0.9975

Epoch 228/1000                                         
252/252 - 1s - loss: 0.4070 - accuracy: 0.9906 - val_loss: 0.2451 - val_accuracy: 0.9955

Epoch 229/1000                                         
252/252 - 1s - loss: 0.3366 - accuracy: 0.9922 - val_loss: 0.3364 - val_accuracy: 0.9975

Epoch 230/1000                    

...

Epoch 25/1000                                                                            
252/252 - 1s - loss: 365.7884 - accuracy: 0.9339 - val_loss: 360.1037 - val_accuracy: 0.9732

Epoch 26/1000                                                                            
252/252 - 1s - loss: 366.8619 - accuracy: 0.9364 - val_loss: 365.9885 - val_accuracy: 0.8867

Epoch 27/1000                                                                            
252/252 - 1s - loss: 367.5619 - accuracy: 0.9391 - val_loss: 362.6425 - val_accuracy: 0.9274

Epoch 28/1000                                                                            
252/252 - 1s - loss: 371.1235 - accuracy: 0.9151 - val_loss: 368.6013 - val_accuracy: 0.9980

Epoch 29/1000                                                                            
252/252 - 1s - loss: 369.5571 - accuracy: 0.9221 - val_loss: 381.6943 - val_accuracy: 0.7177

Epoch 30/1000                                                                   

252/252 - 1s - loss: 370.9229 - accuracy: 0.9412 - val_loss: 367.6824 - val_accuracy: 0.9970

Epoch 70/1000                                                                            
252/252 - 1s - loss: 376.4250 - accuracy: 0.8919 - val_loss: 364.8018 - val_accuracy: 0.9935

Epoch 71/1000                                                                            
252/252 - 1s - loss: 373.4452 - accuracy: 0.8995 - val_loss: 366.4758 - val_accuracy: 0.9901

Epoch 72/1000                                                                            
252/252 - 1s - loss: 372.3829 - accuracy: 0.9147 - val_loss: 365.9799 - val_accuracy: 0.9916

Epoch 73/1000                                                                            
252/252 - 1s - loss: 370.9886 - accuracy: 0.9356 - val_loss: 364.0776 - val_accuracy: 0.9935

Epoch 74/1000                                                                            
252/252 - 1s - loss: 369.8558 - accuracy: 0.9414 - val_loss: 364.0525 - val_accu

Epoch 114/1000                                                                           
252/252 - 1s - loss: 378.9131 - accuracy: 0.7640 - val_loss: 365.8422 - val_accuracy: 0.9612

Epoch 115/1000                                                                           
252/252 - 1s - loss: 375.4085 - accuracy: 0.8104 - val_loss: 370.1298 - val_accuracy: 0.9677

Epoch 116/1000                                                                           
252/252 - 1s - loss: 380.6583 - accuracy: 0.7546 - val_loss: 367.6817 - val_accuracy: 0.9990

Epoch 117/1000                                                                           
252/252 - 1s - loss: 380.0660 - accuracy: 0.7580 - val_loss: 366.2587 - val_accuracy: 0.9826

Epoch 118/1000                                                                           
252/252 - 1s - loss: 392.1005 - accuracy: 0.6018 - val_loss: 370.3665 - val_accuracy: 0.9871

Epoch 119/1000                                                                  

252/252 - 1s - loss: 374.2902 - accuracy: 0.7368 - val_loss: 370.7643 - val_accuracy: 0.8027

Epoch 159/1000                                                                           
252/252 - 1s - loss: 375.3472 - accuracy: 0.7252 - val_loss: 372.3543 - val_accuracy: 0.8971

Epoch 160/1000                                                                           
252/252 - 1s - loss: 373.8782 - accuracy: 0.7564 - val_loss: 370.2321 - val_accuracy: 0.9911

Epoch 161/1000                                                                           
252/252 - 1s - loss: 373.1820 - accuracy: 0.7647 - val_loss: 366.3350 - val_accuracy: 0.8489

Epoch 162/1000                                                                           
252/252 - 1s - loss: 373.7776 - accuracy: 0.7622 - val_loss: 367.7890 - val_accuracy: 0.8181

Epoch 163/1000                                                                           
252/252 - 1s - loss: 373.4636 - accuracy: 0.7668 - val_loss: 365.9948 - val_accu

KeyboardInterrupt: 

In [ ]:
print("Done!!!!!!!!!!!!!!!!!")